In [2]:
%env CUDA_VISIBLE_DEVICES=1
tensor_parallel_size = 1

env: CUDA_VISIBLE_DEVICES=1


In [3]:
ASYNC = False
USE_RAY = False

In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

device = "cpu"
model_path = "/models/huggingface/mistralai/Mistral-7B-Instruct-v0.2"  # "/models/huggingface/meta-llama/llama-2-7b-chat-hf"
model = AutoModelForCausalLM.from_pretrained(
    model_path, device_map=device, torch_dtype=torch.bfloat16
).eval()
tokenizer = AutoTokenizer.from_pretrained(model_path)
embed = model.get_input_embeddings()
if not tokenizer.pad_token:
    if tokenizer.eos_token:
        tokenizer.add_special_tokens(
            {
                "pad_token": tokenizer.eos_token,
            }
        )
    else:
        tokenizer.add_special_tokens({"pad_token": "<PAD>"})
        model.resize_token_embeddings(model.config.vocab_size + 1)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [4]:
from vllm import LLM, SamplingParams, AsyncLLMEngine, AsyncEngineArgs

if ASYNC:
    llm = AsyncLLMEngine.from_engine_args(
        AsyncEngineArgs(
            model="/models/huggingface/mistralai/Mistral-7B-Instruct-v0.2",
            tensor_parallel_size=tensor_parallel_size,
            distributed_executor_backend='ray' if USE_RAY else 'mp',
        )
    )
else:
    llm = LLM(model="/models/huggingface/mistralai/Mistral-7B-Instruct-v0.2",
          distributed_executor_backend='ray' if USE_RAY else 'mp',
          tensor_parallel_size=tensor_parallel_size,
        #   enable_chunked_prefill=True,
        #   max_num_batched_tokens=10,
        #   max_num_seqs=5,
    )

INFO 10-23 16:14:24 llm_engine.py:242] Initializing an LLM engine (v0.6.1.post3.dev539+g3b22bbcb) with config: model='/models/huggingface/mistralai/Mistral-7B-Instruct-v0.2', speculative_config=None, tokenizer='/models/huggingface/mistralai/Mistral-7B-Instruct-v0.2', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=32768, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_time=False, collect_model_execute_time=False), seed=0, served_model_name=/models/huggingface/mistralai/Mistral-7B

Loading safetensors checkpoint shards:   0% Completed | 0/3 [00:00<?, ?it/s]


INFO 10-23 16:14:29 model_runner.py:1116] Loading model weights took 13.4966 GB
INFO 10-23 16:14:33 worker.py:260] Memory profiling results: total_gpu_memory=47.44GiB initial_memory_usage=13.82GiB peak_torch_memory=17.13GiB memory_usage_post_profile=13.83Gib non_torch_memory=0.33GiB kv_cache_size=25.24GiB gpu_memory_utilization=0.90
INFO 10-23 16:14:33 distributed_gpu_executor.py:57] # GPU blocks: 12921, # CPU blocks: 2048
INFO 10-23 16:14:33 distributed_gpu_executor.py:61] Maximum concurrency for 32768 tokens per request: 6.31x
INFO 10-23 16:14:35 model_runner.py:1444] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 10-23 16:14:35 model_runner.py:1448] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce t

In [5]:
prompts = [
    "Hello, my name is",
    "The president of the United States is",
    "My favorite book of all time is",
    "When I wake up in the morning, the first thing I do is",
    "The best vacation I ever took was to",
    "If I could have any superpower, it would be",
    "One thing on my bucket list is",
    "My go-to comfort food is",
    "The most inspiring person in history, in my opinion, is",
    "When I was a child, I wanted to grow up to be",
    "If I could live in any period of history, I would choose",
    "A skill I've always wanted to learn but haven't yet is",
    "The most memorable movie quote for me is",
    "In my free time, I like to",
    "If I could meet any fictional character, I would choose",
    "The last dream I remember having was about",
    "My favorite season of the year is",
    "One thing I wish more people knew about me is",
    "The most delicious meal I've ever had was",
    "If I could travel anywhere in the world right now, I would go to",
    "A hobby I have that most people don't know about is",
    "The best piece of advice I've ever received is",
]
sampling_params = SamplingParams(temperature=0.8, top_p=0.95, max_tokens=100, logprobs=10)

NameError: name 'SamplingParams' is not defined

In [6]:
from pprint import pprint
inputs_embeds = []
for prompt in prompts:
    input_ids = tokenizer.encode(prompt, return_tensors="pt").to(model.device)
    with torch.no_grad():
        input_embeds = embed(input_ids).squeeze(0)
    pprint(input_embeds.shape)
    inputs_embeds.append(input_embeds)

torch.Size([6, 4096])
torch.Size([8, 4096])
torch.Size([8, 4096])
torch.Size([15, 4096])
torch.Size([9, 4096])
torch.Size([12, 4096])
torch.Size([8, 4096])
torch.Size([8, 4096])
torch.Size([14, 4096])
torch.Size([14, 4096])
torch.Size([14, 4096])
torch.Size([16, 4096])
torch.Size([9, 4096])
torch.Size([9, 4096])
torch.Size([13, 4096])
torch.Size([9, 4096])
torch.Size([8, 4096])
torch.Size([11, 4096])
torch.Size([11, 4096])
torch.Size([16, 4096])
torch.Size([15, 4096])
torch.Size([12, 4096])


In [7]:
from uuid import uuid4

if ASYNC:
    results = []
    for prompt in prompts:
        async for request_output in llm.generate(prompt, sampling_params, str(uuid4())):
            pass
        results.append(request_output)
else:
    results = llm.generate(prompts, sampling_params)
pprint(results)

Processed prompts: 100%|██████████| 22/22 [00:02<00:00,  8.01it/s, est. speed input: 89.21 toks/s, output: 801.10 toks/s]

[RequestOutput(request_id=0, prompt='Hello, my name is', prompt_token_ids=[1, 22557, 28725, 586, 1141, 349], prompt_embeds_shape=None, encoder_prompt=None, encoder_prompt_token_ids=None, prompt_logprobs=None, outputs=[CompletionOutput(index=0, text=" Marlene and I'm 56 years old.\n\nI have been a member of the Quirky Quilters Guild since 2014. I have enjoyed meeting so many wonderful people through the Guild and I have learned so much from them.\n\nI live in Clyde, Texas, with my husband, 2 teenage daughters, and 3 dogs. We have a small farm where we raise cattle, goats, chickens, and", token_ids=(1471, 28621, 304, 315, 28742, 28719, 28705, 28782, 28784, 1267, 1571, 28723, 13, 13, 28737, 506, 750, 264, 4292, 302, 272, 2332, 361, 4845, 2332, 309, 1532, 420, 1678, 1854, 28705, 28750, 28734, 28740, 28781, 28723, 315, 506, 10306, 5283, 579, 1287, 8590, 905, 1059, 272, 420, 1678, 304, 315, 506, 5996, 579, 1188, 477, 706, 28723, 13, 13, 28737, 2943, 297, 334, 346, 450, 28725, 7826, 28725, 39

In [8]:
if ASYNC:
    results = []
    for input_embeds in inputs_embeds:
        async for request_output in llm.generate({"prompt_embeds": input_embeds}, sampling_params, str(uuid4())):
            pass
        results.append(request_output)
else:
    results = llm.generate(
        [{"prompt_embeds": input_embeds} for input_embeds in inputs_embeds], sampling_params
    )
pprint(results)

Processed prompts:   0%|          | 0/22 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

AssertionError: 

In [10]:
from random import shuffle

mixed_inputs = [{"prompt_embeds": input_embeds} for input_embeds in inputs_embeds[:10]] + prompts[10:]
shuffle(mixed_inputs)
if ASYNC:
    results = []
    for mixed_input in mixed_inputs:
        async for request_output in llm.generate(mixed_input, sampling_params, str(uuid4())):
            pass
        results.append(request_output)
else:
    results = llm.generate(
        mixed_inputs,
        sampling_params,
    )
pprint(results)

Processed prompts: 100%|██████████| 22/22 [00:02<00:00,  8.84it/s, est. speed input: 57.47 toks/s, output: 884.16 toks/s]

[RequestOutput(request_id=45, prompt='If I could live in any period of history, I would choose', prompt_token_ids=[1, 1047, 315, 829, 2943, 297, 707, 3216, 302, 3340, 28725, 315, 682, 4987], prompt_embeds_shape=None, encoder_prompt=None, encoder_prompt_token_ids=None, prompt_logprobs=None, outputs=[CompletionOutput(index=0, text=' to live during the time of the ancient Greeks. I know that living in ancient Greece would be quite challenging because there would be no modern conveniences or amenities, but I am drawn to their culture and way of life. Here are a few reasons why I would like to live during the time of the ancient Greeks:\n\n1. Philosophy: The ancient Greeks are known for their philosophical contributions to Western civilization. I would love to study under great philosophers like Socr', token_ids=(298, 2943, 1938, 272, 727, 302, 272, 9467, 15752, 2285, 28723, 315, 873, 369, 3687, 297, 9467, 20438, 682, 347, 3448, 14361, 1096, 736, 682, 347, 708, 4638, 8818, 12923, 442, 22891

In [1]:
%%sh

curl http://localhost:8765/v1/models

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   468  100   468    0     0   228k      0 --:--:-- --:--:-- --:--:--  228k


{"object":"list","data":[{"id":"llama3.1","object":"model","created":1731009329,"owned_by":"vllm","root":"llama3.1","parent":null,"max_model_len":131072,"permission":[{"id":"modelperm-b523a92521a14ea08ac0c09c1dbffbf4","object":"model_permission","created":1731009329,"allow_create_engine":false,"allow_sampling":true,"allow_logprobs":true,"allow_search_indices":false,"allow_view":true,"allow_fine_tuning":false,"organization":"*","group":null,"is_blocking":false}]}]}

In [10]:
import requests

url = "http://localhost:8765/v1/completions"
request_model = "llama3.1"

requests.post(
    url,
    json={
        "model": request_model,
        "prompt": prompts,
    },

).json()

{'id': 'cmpl-29dc0cc0210d4daf8b5fbc6b88b34f43',
 'object': 'text_completion',
 'created': 1731009768,
 'model': 'llama3.1',
 'choices': [{'index': 0,
   'text': " Helen. I'm still relatively new to the community and I'm excited to start",
   'logprobs': None,
   'finish_reason': 'length',
   'stop_reason': None,
   'prompt_logprobs': None},
  {'index': 1,
   'text': ' under intense scrutiny these days, mainly because of Robert Mueller’s report on Russian interference',
   'logprobs': None,
   'finish_reason': 'length',
   'stop_reason': None,
   'prompt_logprobs': None},
  {'index': 2,
   'text': ' "Pride and Prejudice" by Jane Austen. I\'ve read',
   'logprobs': None,
   'finish_reason': 'length',
   'stop_reason': None,
   'prompt_logprobs': None},
  {'index': 3,
   'text': ' take a quick shower to get ready for the day. But, sometimes when I',
   'logprobs': None,
   'finish_reason': 'length',
   'stop_reason': None,
   'prompt_logprobs': None},
  {'index': 4,
   'text': ' Tokyo, Ja

In [11]:
import io
from base64 import b64encode
import requests

prompt_embeds = []
for input_embeds in inputs_embeds:
    buff = io.BytesIO()
    torch.save(input_embeds.detach().cpu(), buff)
    prompt_embeds.append(b64encode(buff.getvalue()).decode("utf-8"))

requests.post(
    url,
    json={
        "model": request_model,
        "prompt_embeds": prompt_embeds[0],
        "logprobs": 1,
    },
).json()

{'id': 'cmpl-fdd62d42cdea433f95aa817b8fc0033e',
 'object': 'text_completion',
 'created': 1731009775,
 'model': 'llama3.1',
 'choices': [{'index': 0,
   'text': ' until reverse),%\\&\\ bànolerance items(date=$(date|Wie-im602',
   'logprobs': {'text_offset': [0,
     6,
     14,
     16,
     18,
     19,
     20,
     24,
     32,
     38,
     43,
     46,
     50,
     51,
     54,
     57],
    'token_logprobs': [-12.534470558166504,
     -10.961448669433594,
     -7.97821044921875,
     -12.779780387878418,
     -6.129467964172363,
     -2.764946937561035,
     -14.533710479736328,
     -11.704024314880371,
     -10.93980598449707,
     -11.591647148132324,
     -10.08786392211914,
     -1.8151624202728271,
     -6.723433017730713,
     -12.425421714782715,
     -11.092949867248535,
     -10.94377326965332],
    'tokens': [' until',
     ' reverse',
     '),',
     '%\\',
     '&',
     '\\',
     ' bàn',
     'olerance',
     ' items',
     '(date',
     '=$(',
     'date',
     '

In [12]:
import io
from base64 import b64encode
import requests

prompt_embeds = []
for input_embeds in inputs_embeds:
    buff = io.BytesIO()
    torch.save(input_embeds.detach().cpu(), buff)
    prompt_embeds.append(b64encode(buff.getvalue()).decode("utf-8"))

requests.post(
    url,
    json={
        "model": request_model,
        "prompt_embeds": prompt_embeds[0],
        "best_of": 5,
        # "echo": True,
        "frequency_penalty": 0.5,
        "logit_bias": {12345: 10},
        "logprobs": 1,
        # "prompt_logprobs": 1,
        "max_tokens": 100,
        "n": 2,
        "presence_penalty": 0.5,
        "seed": 42,
        "stop": "<|endoftext|>",
        "temperature": 0.8,
        "top_p": 0.95,
        "user": "string",
        "use_beam_search": False,
        "top_k": -1,
        "min_p": 0,
        "repetition_penalty": 1,
        "length_penalty": 1,
        "early_stopping": False,
        "stop_token_ids": [
            0
        ],
        "add_special_tokens": True,
        "spaces_between_special_tokens": True,
        "truncate_prompt_tokens": 1,
        "response_format": {
            "type": "text",
            "json_schema": {
            "name": "string",
            "description": "string",
            "schema": {},
            "strict": True
            }
        },
    },
).json()

{'id': 'cmpl-bdce354685d947a4863b2723573c6f9c',
 'object': 'text_completion',
 'created': 1731009780,
 'model': 'llama3.1',
 'choices': [{'index': 0,
   'text': '',
   'logprobs': {'text_offset': [0],
    'token_logprobs': [-0.11028390377759933],
    'tokens': [''],
    'top_logprobs': [{'': -0.11028390377759933}]},
   'finish_reason': 'stop',
   'stop_reason': None,
   'prompt_logprobs': None},
  {'index': 1,
   'text': '',
   'logprobs': {'text_offset': [0],
    'token_logprobs': [-0.11028390377759933],
    'tokens': [''],
    'top_logprobs': [{'': -0.11028390377759933}]},
   'finish_reason': 'stop',
   'stop_reason': None,
   'prompt_logprobs': None}],
 'usage': {'prompt_tokens': 0, 'total_tokens': 2, 'completion_tokens': 2}}

In [ ]:
{
  "model": "string",
  "prompt": [
    0
  ],
  "prompt_embeds": "string",
  "best_of": 0,
  "echo": false,
  "frequency_penalty": 0,
  "logit_bias": {
    "additionalProp1": 0,
    "additionalProp2": 0,
    "additionalProp3": 0
  },
  "logprobs": 0,
  "max_tokens": 16,
  "n": 1,
  "presence_penalty": 0,
  "seed": 0,
  "stop": "string",
  "stream": false,
  "stream_options": {
    "include_usage": true,
    "continuous_usage_stats": true
  },
  "suffix": "string",
  "temperature": 1,
  "top_p": 1,
  "user": "string",
  "use_beam_search": false,
  "top_k": -1,
  "min_p": 0,
  "repetition_penalty": 1,
  "length_penalty": 1,
  "early_stopping": false,
  "stop_token_ids": [
    0
  ],
  "include_stop_str_in_output": false,
  "ignore_eos": false,
  "min_tokens": 0,
  "skip_special_tokens": true,
  "spaces_between_special_tokens": true,
  "truncate_prompt_tokens": 0,
  "allowed_token_ids": [
    0
  ],
  "prompt_logprobs": 0,
  "add_special_tokens": true,
  "response_format": {
    "type": "text",
    "json_schema": {
      "name": "string",
      "description": "string",
      "schema": {},
      "strict": true
    }
  },
  "guided_json": "string",
  "guided_regex": "string",
  "guided_choice": [
    "string"
  ],
  "guided_grammar": "string",
  "guided_decoding_backend": "string",
  "guided_whitespace_pattern": "string"
}


In [ ]:
docker run --rm -v /models:/models --gpus '"device=1,3"' -p 8765:8765 vllm-openai:embedding --model /models/meta-llama/llama_3.1  --served-model-name llama3.1 --tensor-parallel-size=2 --port 8765 --host 0.0.0.0

In [15]:
import io
from base64 import b64encode
import requests

prompt_embeds = []
for input_embeds in inputs_embeds:
    buff = io.BytesIO()
    torch.save(input_embeds.detach().cpu(), buff)
    prompt_embeds.append(b64encode(buff.getvalue()).decode("utf-8"))

requests.post(
    url,
    json={
        "model": request_model,
        "prompt_embeds": prompt_embeds,
    },
).json()

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [17]:
requests.post(
    url,
    json={
        "model": request_model,
        "prompt": prompts,
        "prompt_embeds": prompt_embeds,
    },
).json()

{'id': 'cmpl-b9be72dd774548b0819c3d428813acad',
 'object': 'text_completion',
 'created': 1731010259,
 'model': 'llama3.1',
 'choices': [{'index': 0,
   'text': ' Rachel Nolan! I am a freelance interior designer based in San Francisco, California.',
   'logprobs': None,
   'finish_reason': 'length',
   'stop_reason': None,
   'prompt_logprobs': None},
  {'index': 1,
   'text': ' the head of the U.S. federal government. He is the highest-ranking individual',
   'logprobs': None,
   'finish_reason': 'length',
   'stop_reason': None,
   'prompt_logprobs': None},
  {'index': 2,
   'text': ' the deceptively plain, iconic\xa0 Manslaughter or Endtime, Ending in',
   'logprobs': None,
   'finish_reason': 'length',
   'stop_reason': None,
   'prompt_logprobs': None},
  {'index': 3,
   'text': ' stretch, then pick up my pineapple slice, and check my notifications. Today isn',
   'logprobs': None,
   'finish_reason': 'length',
   'stop_reason': None,
   'prompt_logprobs': None},
  {'index': 4,
  

In [ ]:
import requests

url = "http://localhost:8000/v1/completions"

response = requests.post(
    url,
    json={
        "model": request_model,
        "prompt_embeds": prompt_embeds,
    },
    stream=True,
)

In [14]:
from typing import Any
import aiohttp


async with aiohttp.ClientSession() as session:
    async with session.post(
        url,
        json={
            "model": request_model,
            "prompt_embeds": prompt_embeds[0],
            "max_tokens": 10,
            "stream": True,
        },
    ) as resp:
        chunks = []
        async for chunk, _ in resp.content.iter_chunks():
            chunk = chunk.decode()
            chunks.append(chunk)
            print(chunk)

data: {"id":"cmpl-4a0f1617bf224b18b43457cd25e18ecc","object":"text_completion","created":1730690214,"model":"mistral-7b","choices":[{"index":0,"text":"Sarah","logprobs":null,"finish_reason":null,"stop_reason":null}],"usage":null}


data: {"id":"cmpl-4a0f1617bf224b18b43457cd25e18ecc","object":"text_completion","created":1730690214,"model":"mistral-7b","choices":[{"index":0,"text":",","logprobs":null,"finish_reason":null,"stop_reason":null}],"usage":null}


data: {"id":"cmpl-4a0f1617bf224b18b43457cd25e18ecc","object":"text_completion","created":1730690214,"model":"mistral-7b","choices":[{"index":0,"text":" and","logprobs":null,"finish_reason":null,"stop_reason":null}],"usage":null}


data: {"id":"cmpl-4a0f1617bf224b18b43457cd25e18ecc","object":"text_completion","created":1730690214,"model":"mistral-7b","choices":[{"index":0,"text":" I","logprobs":null,"finish_reason":null,"stop_reason":null}],"usage":null}


data: {"id":"cmpl-4a0f1617bf224b18b43457cd25e18ecc","object":"text_completion","

In [16]:
chunks


['data: {"id":"cmpl-4a0f1617bf224b18b43457cd25e18ecc","object":"text_completion","created":1730690214,"model":"mistral-7b","choices":[{"index":0,"text":"Sarah","logprobs":null,"finish_reason":null,"stop_reason":null}],"usage":null}\n\n',
 'data: {"id":"cmpl-4a0f1617bf224b18b43457cd25e18ecc","object":"text_completion","created":1730690214,"model":"mistral-7b","choices":[{"index":0,"text":",","logprobs":null,"finish_reason":null,"stop_reason":null}],"usage":null}\n\n',
 'data: {"id":"cmpl-4a0f1617bf224b18b43457cd25e18ecc","object":"text_completion","created":1730690214,"model":"mistral-7b","choices":[{"index":0,"text":" and","logprobs":null,"finish_reason":null,"stop_reason":null}],"usage":null}\n\n',
 'data: {"id":"cmpl-4a0f1617bf224b18b43457cd25e18ecc","object":"text_completion","created":1730690214,"model":"mistral-7b","choices":[{"index":0,"text":" I","logprobs":null,"finish_reason":null,"stop_reason":null}],"usage":null}\n\n',
 'data: {"id":"cmpl-4a0f1617bf224b18b43457cd25e18ecc","o

In [37]:
from openai import OpenAI
client = OpenAI(
    base_url="http://localhost:8000/v1", api_key="secret"
)

completion = client.chat.completions.create(
  model="/models/huggingface/mistralai/Mistral-7B-Instruct-v0.2",
  messages=[
    {"role": "user", "content": "Hello!"}
  ]
)

print(completion.choices[0].message)

ChatCompletionMessage(content=" Hello! How can I help you today? I'm here to answer any questions you might have, provide information, or just chat if you'd like. Let me know what's on your mind!\n\nHere are some topics I can help with:\n\n1. Answering factual questions\n2. Providing definitions and explanations\n3. Offering suggestions for movies, TV shows, books, and other forms of entertainment\n4. Helping with math problems and other academic queries\n5. Providing advice on various topics, such as relationships, career, and personal development\n\nSo, what would you like to talk about today? Let me know and I'll do my best to make our conversation enjoyable and productive!", refusal=None, role='assistant', function_call=None, tool_calls=[])


In [41]:
completion = client.completions.create(
    model="/models/huggingface/mistralai/Mistral-7B-Instruct-v0.2",
    prompt="Hello!",
)
print(completion)

Completion(id='cmpl-9d8b04dcca6d46338beb1bb31b4a9d82', choices=[CompletionChoice(finish_reason='length', index=0, logprobs=None, text=" I'm here waving at you this weekend because we're having a", stop_reason=None, prompt_logprobs=None)], created=1729263312, model='/models/huggingface/mistralai/Mistral-7B-Instruct-v0.2', object='text_completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=16, prompt_tokens=3, total_tokens=19))


In [40]:
completion = client.completions.create(
    model="/models/huggingface/mistralai/Mistral-7B-Instruct-v0.2",
    prompt="Hello!",
)
print(completion)

1

In [2]:
from typing import Optional
import torch

def get_inputs_embeds(
    input_ids: torch.Tensor,
    embeddings_module: torch.nn.Module,
    inputs_embeds: Optional[torch.Tensor] = None,
    inputs_embeds_masks: Optional[torch.Tensor] = None,
) -> torch.Tensor:
    """Get the input embeddings from either `input_ids` and `inputs_embeds`."""
    if inputs_embeds is not None:
        if inputs_embeds_masks.all().item():
            hidden_states = inputs_embeds
        else:
            hidden_states = embeddings_module(input_ids)
            hidden_states[inputs_embeds_masks] = inputs_embeds
    else:
        hidden_states = embeddings_module(input_ids)
    return hidden_states


def get_inputs_embeds_new(
    input_ids: torch.Tensor,
    positions: torch.Tensor,
    embeddings_module: torch.nn.Module,
    inputs_embeds: Optional[torch.Tensor] = None,
    inputs_embeds_masks: Optional[torch.Tensor] = None,
) -> torch.Tensor:
    """Get the input embeddings from either `input_ids` and `inputs_embeds`."""
    if inputs_embeds is not None:
        if inputs_embeds_masks.all().item():
            hidden_states = inputs_embeds
        else:
            hidden_states = embeddings_module(input_ids)
            starting_positions = [idx for idx, v in enumerate(positions) if v == 0] + [len(positions)]
            embeds_position = 0
            for idx, has_embeds in enumerate(inputs_embeds_masks):
                if has_embeds:
                    seq_len = starting_positions[idx + 1] - starting_positions[idx]
                    hidden_states[starting_positions[idx]:starting_positions[idx + 1]] = inputs_embeds[embeds_position:embeds_position + seq_len]
                    embeds_position += seq_len
    else:
        hidden_states = embeddings_module(input_ids)
    return  hidden_states

In [10]:
embed.to("cuda")
embed.weight.device

device(type='cuda', index=0)

In [23]:
input_positions = torch.arange(0, 10, device="cuda").repeat(10)
input_ids = input_positions
input_embeds = embed(input_ids[:50])
input_embeds_masks = torch.zeros_like(input_ids, dtype=torch.bool, device="cuda")
input_embeds_masks[:50] = True
input_embeds_masks_new = torch.zeros(10, dtype=torch.bool, device="cuda")
input_embeds_masks_new[:5] = True

In [31]:
%timeit get_inputs_embeds(input_ids, embed, input_embeds, input_embeds_masks)
%timeit get_inputs_embeds_new(input_ids, input_positions, embed, input_embeds, input_embeds_masks_new)

84.8 μs ± 103 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
1.96 ms ± 3.44 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [27]:
get_inputs_embeds_new(input_ids, input_positions, embed, input_embeds, input_embeds_masks_new).shape

torch.Size([100, 4096])

In [16]:
import torch

x = torch.zeros(2, dtype=torch.bool)
x

tensor([False, False])

In [17]:
x.fill_(1)

tensor([True, True])

In [ ]:
x